<font color='steelblack'>
    <h1 align=center> Analyse de sentiments (NLP) via 3 approches :</h1>
    <h2 align=center>Sklearn simple, Pytorch simple, Pytorch-LSTM TORCH </h2>
          </font>

---

<h2 align=center>Cours conférence de Sébastien Collet - Data Scientist chez Saagie : Orchestrateur pour la DataOps</h2>

<h3 align=center>Jean Martial Tagro - Data Scientist</h3>

Il s'agit de l'analyse d'un dataset de sentiments composé de quelques millions d'avis clients Amazon (texte d'entrée) et d'étoiles (étiquettes de sortie).
Ce dataset constitue de vraies données commerciales à une échelle raisonnable mais peut être appris en un temps relativement court sur un ordinateur portable modeste. Dans le dataset, label 1 : sentiment positif ; label 2 : sentiment négatif.<br>
Source : voir <a href='https://www.kaggle.com/bittlingmayer/amazonreviews?select=test.ft.txt.bz2'>Kaggle</a>
<br>Evidement, les 3 approches étudiées dans ce Notebook sont indépendantes.

In [1]:
#pip install torch torchvision

In [109]:
# Librairies
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
import torch.tensor as tensor
import torch.cuda as cuda
from torch.utils.data import TensorDataset, DataLoader

In [148]:
# Importation données source
# engine='python' to take account sep > 1 char
data_train = pd.read_csv('sentiment-train.txt', sep='##label##', header=None, names=['label','text'], engine='python')
data_test  = pd.read_csv('sentiment-test.txt', sep='##label##', header=None, names=['label', 'text'], engine='python')
data_train.shape, data_test.shape

((3600000, 2), (400000, 2))

---

<font color='darkred'>
    <h1 align=center>Partie 1 : Approche Sklearn simple</h1>
          </font>

## 1. Préparation des données

#### Vu la taille des données textuelles (4 millions de commentaires au total), nous allons dans un 1er temps faire l'étude sur un échantillon.

Regardons les proportions de label avant le sampling :

In [149]:
print(np.unique(data_train['label'].values, return_counts=True))
print(np.unique(data_test['label'].values, return_counts=True))

(array([1, 2]), array([1800000, 1800000]))
(array([1, 2]), array([200000, 200000]))


Les proportions de sentiments positifs (2) et négatifs (1) sont égales. Obtenons de même un échantillons du dataset à proportions équivalentes de sentiments :

In [150]:
# Sampling (avec 1 entrée sur 100)

data_train_sample = data_train.sample(frac=1/100)
data_test_sample = data_test.sample(frac=1/100)

#### Useful - from Stack Overflow
df.sample(frac=1)
The frac keyword argument specifies the fraction of rows to return in the random sample, so frac=1 means return all rows (in random order).

Note: If you wish to shuffle your dataframe in-place and reset the index, you could do e.g.

df = df.sample(frac=1).reset_index(drop=True)
Here, specifying drop=True prevents .reset_index from creating a column containing the old index entries.

In [151]:
# Train test Split
X_train = data_train_sample['text'].values
y_train = data_train_sample['label'].values

X_test = data_test_sample['text'].values
y_test = data_test_sample['label'].values

---

## 2. Pre-processing des corpus de texte

In [152]:
# Creation des vecteurs One Hot : Transform mots en features + Bag of Words (...)
count_vect = CountVectorizer()
X_train_count = count_vect.fit_transform(X_train)
X_train_count

<36000x68286 sparse matrix of type '<class 'numpy.int64'>'
	with 1972308 stored elements in Compressed Sparse Row format>

#### Enorme matrice ! 36000 commentaires représentés par un vecteur de taille 68422 !
#### Heureusement que l'objet Sparse matrix stocke en mémoire de façon plus intelligence des gros objets où il y a beaucoup de zéros.

In [153]:
X_train_count.shape

(36000, 68286)

### --> Pour une question de test rapide on va garder les milles features les plus fréquents :

In [154]:
# Creation des vecteurs One Hot : Transform mots en features + Bag of Words (...) -- avec une feature vector de taille 1000
count_vect = CountVectorizer(max_features=1000)
X_train_count = count_vect.fit_transform(X_train)
X_train_count

<36000x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 1437131 stored elements in Compressed Sparse Row format>

---

## 3. Implémentation du modèle

In [155]:
classifier = LogisticRegression()

classifier.fit(X_train_count, y_train)

/Users/jean-martial/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## 4. Prédiction sur le jeu de test

In [156]:
# Transformations prealables
X_test_count = count_vect.transform(X_test)  # Ne surtout pas (re)faire fit (...)

In [159]:
print('Accuracy score : {:.2f} %'.format(100*classifier.score(X_test_count, y_test)))

Accuracy score : 86.05 %


---

<font color='darkgreen'>
    <h1 align=center>Partie 2 : Approche PyTorch simple</h1>
          </font>

## 1. Préparation des données

In [170]:
# Sampling (avec 1 entrée sur 100)

data_train_sample = data_train.sample(frac=1/100)
data_test_sample = data_test.sample(frac=1/100)

In [171]:
# Train test Split
X_train = data_train_sample['text'].values
y_train = data_train_sample['label'].values

X_test = data_test_sample['text'].values
y_test = data_test_sample['label'].values

In [172]:
# Remplacement des labels 1-2 par 0-1
y_train = np.where(y_train == 1, 0, 1)
y_test = np.where(y_test == 1, 0, 1)

## 2. Pre-processing des corpus de texte

In [173]:
# Creation des vecteurs One Hot : Transform mots en features + Bag of Words (...) -- avec une feature vector de taille 1000
count_vect = CountVectorizer(max_features=1000)
X_train_count = count_vect.fit_transform(X_train)
X_test_count = count_vect.transform(X_test)

X_train_count

<36000x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 1440155 stored elements in Compressed Sparse Row format>

## 3. Transformation des arrays numpy en tensors

In [174]:
tensor_X_train = tensor(X_train_count.toarray(), dtype=torch.float)
tensor_y_train = tensor(y_train, dtype=torch.long)

tensor_X_test = tensor(X_test_count.toarray(), dtype=torch.float)
tensor_y_test = tensor(y_test, dtype=torch.long)

---

## 4. Implémentation du modèle

In [175]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [176]:
class Perceptron(nn.Module): # nn.Module to let pyTorch know our class is a neural network
    
    def __init__(self):
        super(Perceptron, self).__init__()
        self.fc1 = nn.Linear(tensor_X_train.shape[1], 2)
        #self.activation = nn.Sigmoid()
        
    def forward(self, x):
        x = self.fc1(x)
        return x

In [177]:
classifier = Perceptron().to(device)

In [178]:
# Check out initial parameters
for param in classifier.named_parameters():
    print(param)

('fc1.weight', Parameter containing:
tensor([[ 0.0112,  0.0252, -0.0315,  ..., -0.0284,  0.0121,  0.0236],
        [-0.0066,  0.0059, -0.0177,  ..., -0.0208, -0.0309, -0.0010]],
       requires_grad=True))
('fc1.bias', Parameter containing:
tensor([-0.0043, -0.0295], requires_grad=True))


---

## 5. Définitions : dataloader (batch), loss, optimizer

In [179]:
tensor_trainset = TensorDataset(tensor_X_train, tensor_y_train) # fusion du jeu de train en 1 tenseur (text+label)
dataloader_train = DataLoader(tensor_trainset, batch_size = 32, shuffle = True)

tensor_testset = TensorDataset(tensor_X_test, tensor_y_test) # fusion du jeu de test en 1 tenseur (text+label)
dataloader_test = DataLoader(tensor_testset, batch_size = 32, shuffle = True)

# loss and optimizer definition
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr = 0.01, weight_decay = 0.5)

## 6. Boucle d'apprentissage du modèle

In [180]:
for epoch in range(1,11):
    for batch in dataloader_train:
        texts, labels = batch
        texts, labels = texts.to(device), labels.to(device)
    
        outputs = classifier(texts)
        loss = criterion(outputs, labels)
        optimizer.zero_grad() # to re-initialize gradient (not sum) after each batch
        loss.backward()
        optimizer.step()
                         
    print('epoch {} : loss = {}'.format(epoch, loss))

epoch 1 : loss = 0.5691317319869995
epoch 2 : loss = 0.7858763337135315
epoch 3 : loss = 0.6035353541374207
epoch 4 : loss = 0.5927248001098633
epoch 5 : loss = 0.6165134906768799
epoch 6 : loss = 0.6241710186004639
epoch 7 : loss = 0.7456296682357788
epoch 8 : loss = 0.7697122097015381
epoch 9 : loss = 0.6473281383514404
epoch 10 : loss = 0.6206763982772827


---

## 7. Prédictions sur le jeu de test

In [181]:
all_labels = []
all_preds = []

with torch.no_grad(): # by default, pyTorch create graph to calculate gradients. -> No need here
    for batch in dataloader_test:
        texts, labels = batch
        texts, labels = texts.to(device), labels.to(device)

        outputs = classifier(texts)
        _, predicted = torch.max(outputs, 1) # get the max values of both two output neurons - y axis
        all_preds.append(predicted)
        all_labels.append(labels)

all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)
print('accuracy score = {}'.format(accuracy_score(all_preds, all_labels)))

accuracy score = 0.68075


Ce simple perceptron est moins performant qu'une regression logistique.<br>
Cette faible performance aussi est dûe qu'on a pas de Word Embedding (...)

---

<font color='darkblue'>
    <h1 align=center>Partie 3 : Approche PyTorch-LSTM-TORCHTEXT</h1>
          </font>

In [185]:
# torchtext contient plein de fonctions pour gestion de texte et sequences
# pip install torchtext

In [187]:
# Imports specifiques
import torchtext
from torchtext.data import Field, TabularDataset, BucketIterator

## 1. Définition des pré-traitement sur le texte

In [188]:
# ... >>>